# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

### 2) Config file 만들기 (선택)
- 모델 생성에 필요한 다양한 매개변수 정보를 저장할 수 있습니다.  
  따라서, 코드 상에서 모델의 매개변수를 설정할 수도 있지만 독립적인 매개변수 정보 파일을 생성하여 관리할 수 있습니다.

- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

### 3) Configuration 불러오기

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [1]:
# ===============================
# 1. 데이터 및 환경설정
# ===============================

import os
import re
import json
import yaml
from pprint import pprint

import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer,
    BartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

from rouge import Rouge  # 모델 성능 평가용
from peft import LoraConfig, get_peft_model, PeftModel

import wandb  # 선택: 실험 로깅용


# ---------- (선택) LoRA 관련 패키지 설치: 필요 시 한 번만 실행 ----------
# !pip install peft accelerate bitsandbytes


# ---------- 요약문 후처리 유틸 함수 ----------
def clean_generated_summary(summary_list, remove_tokens):
    """
    생성된 요약문에서 불필요한 토큰들을 제거합니다.
    """
    escaped_tokens = [re.escape(token) for token in remove_tokens]
    pattern = "|".join(escaped_tokens)

    cleaned_texts = []
    for text in summary_list:
        cleaned = re.sub(pattern, " ", text)
        cleaned = re.sub(r"\s+", " ", cleaned)
        cleaned = cleaned.strip()
        cleaned_texts.append(cleaned)

    return cleaned_texts


def remove_repeated_ngrams(text, n=3):
    """
    문장 내에서 반복되는 N-gram을 제거합니다.
    예: "오늘 날씨가 좋아요 오늘 날씨가 좋아요" → "오늘 날씨가 좋아요"
    """
    tokens = text.split()
    seen = set()
    result = []

    for i in range(len(tokens)):
        ngram = tuple(tokens[i:i+n])

        if len(ngram) < n:
            result.append(tokens[i])
            continue

        if ngram not in seen:
            seen.add(ngram)
            result.append(tokens[i])
        else:
            # 중복된 N-gram이면 skip
            continue

    return " ".join(result)


def clean_repetition_in_summaries(summary_list, n=3):
    """
    여러 요약문에 대해 N-gram 반복을 제거하는 wrapper 함수.
    """
    cleaned_list = []
    for text in summary_list:
        cleaned = remove_repeated_ngrams(text, n=n)
        cleaned = re.sub(r"\s+", " ", cleaned).strip()
        cleaned_list.append(cleaned)
    return cleaned_list


# ---------- (선택) wandb 로그인 ----------
# 보안상 API 키는 환경변수나 CLI 로그인 사용을 권장
# wandb.login()   # CLI에서 미리 `wandb login` 한 경우
# 또는
wandb.login(key="624410f236117b79b2ee07a4ccef31c7514a5e03")


# ---------- tokenizer 선 로드 (config 생성용) ----------
base_model_name = "digit82/kobart-summarization"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

config_data = {
    "general": {
        "data_path": "./data/",                # train.csv, dev.csv, test.csv 위치
        "model_name": base_model_name,         # 사용 모델
        "output_dir": "./prediction/",         # 예측 결과 저장 경로
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "special_tokens": [
            "#Person1#",
            "#Person2#",
            "#Person3#",
            "#PhoneNumber#",
            "#Address#",
            "#PassportNumber#",
        ],
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 10,
        "learning_rate": 5e-5,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 8,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": "cosine",
        "optim": "adamw_torch",
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": "epoch",
        "save_strategy": "no",           # HF Trainer 자동 체크포인트 저장 X (LoRA만 별도로 저장)
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": False,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb",             # "wandb" 로 바꾸면 wandb 사용
    },
    "wandb": {
        "entity": "fc_bootcamp",           # 사용자 계정/팀 이름
        "project": "nlp",                # 프로젝트 이름
        "name": "run_name",           # run 이름
    },
    "inference": {
        "ckt_path": "./lora_epoch_20",   # 사용할 LoRA adapter 디렉토리 (학습 후 수정)
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 80,
        "num_beams": 5,
        "batch_size": 32,
        "remove_tokens": [
            "<usr>",
            f"{tokenizer.bos_token}",
            f"{tokenizer.eos_token}",
            f"{tokenizer.pad_token}",
        ],
    },
}

# ---------- YAML로 저장 ----------
os.makedirs("config", exist_ok=True)
config_path = "config/config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

# ---------- YAML 로드 ----------
with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

print("✅ Loaded config keys:")
pprint(list(loaded_config.keys()))


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: quriquri7 (fc_bootcamp). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc
/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Loaded config keys:
['general', 'inference', 'tokenizer', 'training', 'wandb']


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [2]:
# ===============================
# 2. 데이터 가공 및 Dataset 클래스
# ===============================

class Preprocess:
    def __init__(self, bos_token: str, eos_token: str, max_turns: int = 12):
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.max_turns = max_turns

    @staticmethod
    def make_set_as_df(file_path, is_train: bool = True):
        df = pd.read_csv(file_path)
        if is_train:
            return df[["fname", "dialogue", "summary"]]
        else:
            return df[["fname", "dialogue"]]

    def _trim_dialogue(self, text: str) -> str:
        """
        1) 줄바꿈 기준으로 턴 나누기
        2) 마지막 max_turns 개만 사용
        3) 너무 짧은 filler 턴은 제거
        """
        turns = text.split("\n")

        # 마지막 max_turns만 사용
        if len(turns) > self.max_turns:
            turns = turns[-self.max_turns:]

        cleaned = []
        for t in turns:
            t_strip = t.strip()
            if len(t_strip) < 3:
                continue
            # 흔한 filler 몇 개 제거 예시 (필요시 더 추가)
            if t_strip in ["네", "네네", "예", "알겠습니다", "네, 알겠습니다"]:
                continue
            cleaned.append(t_strip)

        return "\n".join(cleaned)

    def make_input(self, dataset, is_test: bool = False):
        dialogues = dataset["dialogue"].apply(self._trim_dialogue)

        if is_test:
            encoder_input = dialogues
            decoder_input = [self.bos_token] * len(dialogues)
            return encoder_input.tolist(), decoder_input
        else:
            encoder_input = dialogues
            decoder_input = dataset["summary"].apply(
                lambda x: self.bos_token + str(x)
            )
            decoder_output = dataset["summary"].apply(
                lambda x: str(x) + self.eos_token
            )
            return (
                encoder_input.tolist(),
                decoder_input.tolist(),
                decoder_output.tolist(),
            )


class Seq2SeqDataset(Dataset):
    """
    Train / Validation 공용 Dataset
    """

    def __init__(self, encoder_input, decoder_input, labels, length: int):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        # encoder
        item = {
            key: val[idx].clone().detach()
            for key, val in self.encoder_input.items()
        }  # input_ids, attention_mask

        # decoder
        dec = {
            key: val[idx].clone().detach()
            for key, val in self.decoder_input.items()
        }  # input_ids, attention_mask
        dec["decoder_input_ids"] = dec["input_ids"]
        dec["decoder_attention_mask"] = dec["attention_mask"]
        dec.pop("input_ids")
        dec.pop("attention_mask")

        item.update(dec)
        item["labels"] = self.labels["input_ids"][idx]  # teacher forcing labels

        return item

    def __len__(self):
        return self.length


class DatasetForInference(Dataset):
    """
    Test용 Dataset (encoder 입력 + fname)
    """

    def __init__(self, encoder_input, test_id, length: int):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.length = length

    def __getitem__(self, idx):
        item = {
            key: val[idx].clone().detach()
            for key, val in self.encoder_input.items()
        }
        item["ID"] = self.test_id[idx]
        return item

    def __len__(self):
        return self.length


def prepare_train_dataset(config, preprocessor: Preprocess, data_path: str, tokenizer):
    """
    train / dev CSV를 읽어서 tokenization까지 완료한 Dataset 반환
    """
    train_file_path = os.path.join(data_path, "train.csv")
    val_file_path = os.path.join(data_path, "dev.csv")

    train_data = preprocessor.make_set_as_df(train_file_path, is_train=True)
    val_data = preprocessor.make_set_as_df(val_file_path, is_train=True)

    # 텍스트 전처리
    encoder_input_train, decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val, decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)

    print("✅ Load raw data complete")

    # tokenization
    tokenized_encoder_inputs = tokenizer(
        encoder_input_train,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config["tokenizer"]["encoder_max_len"],
        return_token_type_ids=False,
    )
    tokenized_decoder_inputs = tokenizer(
        decoder_input_train,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )
    tokenized_decoder_outputs = tokenizer(
        decoder_output_train,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )

    train_dataset = Seq2SeqDataset(
        tokenized_encoder_inputs,
        tokenized_decoder_inputs,
        tokenized_decoder_outputs,
        len(encoder_input_train),
    )

    val_tokenized_encoder_inputs = tokenizer(
        encoder_input_val,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config["tokenizer"]["encoder_max_len"],
        return_token_type_ids=False,
    )
    val_tokenized_decoder_inputs = tokenizer(
        decoder_input_val,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )
    val_tokenized_decoder_outputs = tokenizer(
        decoder_output_val,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )

    val_dataset = Seq2SeqDataset(
        val_tokenized_encoder_inputs,
        val_tokenized_decoder_inputs,
        val_tokenized_decoder_outputs,
        len(encoder_input_val),
    )

    print("✅ Make train / val dataset complete")
    return train_dataset, val_dataset


## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [3]:
# ===============================
# 3. Trainer 및 TrainingArguments
# ===============================

def compute_metrics(config, tokenizer, pred):
    """
    Rouge 기반 평가 지표 계산
    - 전처리 후 빈 문자열이 되는 예측/정답 쌍을 안전하게 처리
    """
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    # -100 → pad_token_id로 복원
    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(
        predictions, clean_up_tokenization_spaces=True
    )
    decoded_labels = tokenizer.batch_decode(
        labels, clean_up_tokenization_spaces=True
    )

    # 불필요한 토큰 제거
    remove_tokens = config["inference"]["remove_tokens"]

    def clean_text_list(text_list):
        out = []
        for s in text_list:
            for token in remove_tokens:
                s = s.replace(token, " ")
            s = re.sub(r"\s+", " ", s).strip()
            out.append(s)
        return out

    replaced_predictions = clean_text_list(decoded_preds)
    replaced_labels = clean_text_list(decoded_labels)

    # --- 빈 문자열 안전 처리 ---
    pairs = []
    for p, l in zip(replaced_predictions, replaced_labels):
        # 예측/정답이 둘 다 완전히 비어 있으면 스킵
        if (not p.strip()) and (not l.strip()):
            continue

        # 예측이 비어 있으면 최소 더미 토큰 하나 넣어줌
        if not p.strip():
            p = "."

        # 정답이 비어 있을 일은 거의 없지만 방어적으로 처리
        if not l.strip():
            l = "."

        pairs.append((p, l))

    if len(pairs) == 0:
        # 전부 비어 있는 극단적 케이스라면 0점 리턴
        return {"rouge-1": 0.0, "rouge-2": 0.0, "rouge-l": 0.0}

    preds_clean, labels_clean = zip(*pairs)
    preds_clean = list(preds_clean)
    labels_clean = list(labels_clean)

    # 모니터링용으로 첫 샘플 출력
    print("-" * 120)
    print(f"PRED[0]: {preds_clean[0]}")
    print(f"GOLD[0]: {labels_clean[0]}")

    # ROUGE 점수 계산 (avg=True)
    results = rouge.get_scores(preds_clean, labels_clean, avg=True)
    result = {key: value["f"] for key, value in results.items()}
    return result


def build_training_args(config):
    """
    HF Trainer 용 TrainingArguments 생성
    - 실제 epoch 루프는 바깥(main)에서 돌리므로 num_train_epochs=1 고정
    """
    return Seq2SeqTrainingArguments(
        output_dir=config["general"]["output_dir"],
        overwrite_output_dir=config["training"]["overwrite_output_dir"],
        num_train_epochs=1,  # 한 번 train() 호출 시 1 epoch만
        learning_rate=config["training"]["learning_rate"],
        per_device_train_batch_size=config["training"]["per_device_train_batch_size"],
        per_device_eval_batch_size=config["training"]["per_device_eval_batch_size"],
        warmup_ratio=config["training"]["warmup_ratio"],
        weight_decay=config["training"]["weight_decay"],
        lr_scheduler_type=config["training"]["lr_scheduler_type"],
        evaluation_strategy=config["training"]["evaluation_strategy"],
        save_strategy=config["training"]["save_strategy"],  # "no"
        save_total_limit=config["training"]["save_total_limit"],
        load_best_model_at_end=config["training"]["load_best_model_at_end"],
        logging_dir=config["training"]["logging_dir"],
        logging_strategy=config["training"]["logging_strategy"],
        predict_with_generate=config["training"]["predict_with_generate"],
        generation_max_length=config["training"]["generation_max_length"],
        fp16=config["training"]["fp16"],
        save_safetensors=True,
        seed=config["training"]["seed"],
        report_to=config["training"]["report_to"],  # "none" 또는 "wandb"
        gradient_accumulation_steps=config["training"]["gradient_accumulation_steps"],
    )


def apply_lora_to_model(model):
    """
    KoBART에 LoRA 주입
    """
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_2_SEQ_LM",
        target_modules=["q_proj", "v_proj"],
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    return model


def load_tokenizer_and_model_for_train(config, device):
    """
    학습용 KoBART + LoRA 모델 로더
    """
    print("---------- Load tokenizer & model with LoRA ----------")

    model_name = config["general"]["model_name"]
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # special tokens 추가
    tokenizer.add_special_tokens(
        {"additional_special_tokens": config["tokenizer"]["special_tokens"]}
    )

    # base BART model load
    model = BartForConditionalGeneration.from_pretrained(model_name)
    model.resize_token_embeddings(len(tokenizer))

    # LoRA 적용
    model = apply_lora_to_model(model)
    model.to(device)

    print("✅ Model config:")
    print(model.config)

    return model, tokenizer


def build_trainer(model, tokenizer, train_ds, val_ds, config):
    """
    Seq2SeqTrainer 생성
    - 내부 save_model은 막고, LoRA adapter는 별도 함수에서 저장
    """
    training_args = build_training_args(config)

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=lambda pred: compute_metrics(config, tokenizer, pred),
    )

    # HF Trainer 기본 save_model 기능 무력화 (shared weight error 방지)
    trainer.save_model = lambda *args, **kwargs: None

    return trainer


def save_lora_adapter(model, epoch: int):
    """
    epoch별 LoRA adapter만 저장
    """
    save_dir = f"./lora_epoch_{epoch}"
    os.makedirs(save_dir, exist_ok=True)
    model.save_pretrained(save_dir)
    print(f"💾 Saved LoRA adapter: {save_dir}")


## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [4]:
# ===============================
# 4. 모델 학습하기
# ===============================

def main(config):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("---------- device:", device, "----------")
    print(torch.__version__)

    # (선택) wandb 활성화 시
    if config["training"]["report_to"] == "wandb":
        wandb.init(
            entity=config["wandb"]["entity"],
            project=config["wandb"]["project"],
            name=config["wandb"]["name"],
        )
        os.environ["WANDB_LOG_MODEL"] = "true"
        os.environ["WANDB_WATCH"] = "false"
    else:
        os.environ["WANDB_DISABLED"] = "true"
        print("🚫 wandb 비활성화 상태로 학습을 진행합니다.")

    # 1) 모델 & 토크나이저 로드 (LoRA 적용)
    model, tokenizer = load_tokenizer_and_model_for_train(config, device)

    # 2) 데이터 로드
    preprocessor = Preprocess(
        config["tokenizer"]["bos_token"], config["tokenizer"]["eos_token"], max_turns=12,  # ← 여기 숫자만 바꿔가며 실험 (10~14 추천)
    )
    train_ds, val_ds = prepare_train_dataset(
        config, preprocessor, config["general"]["data_path"], tokenizer
    )

    # 3) Trainer 생성
    trainer = build_trainer(model, tokenizer, train_ds, val_ds, config)

    # 4) epoch 루프
    total_epoch = config["training"]["num_train_epochs"]

    for epoch in range(total_epoch):
        print(f"\n\n🔥🔥🔥 EPOCH {epoch + 1}/{total_epoch} 시작 🔥🔥🔥")
        trainer.train()
        print(f"🔥 Epoch {epoch + 1} 종료")

        # LoRA adapter 저장
        save_lora_adapter(model, epoch + 1)

    if config["training"]["report_to"] == "wandb":
        wandb.finish()


if __name__ == "__main__":
    main(loaded_config)


---------- device: cuda ----------
2.1.2+cu118


---------- Load tokenizer & model with LoRA ----------


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


trainable params: 884,736 || all params: 124,749,312 || trainable%: 0.7092
✅ Model config:
BartConfig {
  "_name_or_path": "digit82/kobart-summarization",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": t

/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

🔥 Epoch 10 종료


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


💾 Saved LoRA adapter: ./lora_epoch_10


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/loss,█▅▄▃▂▂▂▁▁▁
eval/rouge-1,▁▄▅▆▆▇████
eval/rouge-2,▁▄▅▆▆▇████
eval/rouge-l,▁▄▅▆▆▇████
eval/runtime,█▁▃▅▅▅▄▅▅▆
eval/samples_per_second,▁█▆▄▄▄▅▄▃▃
eval/steps_per_second,▁█▆▄▄▄▅▄▃▃
train/epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/global_step,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,█▁▁▁▁▁▁▁▁▁
train/loss,█▂▂▂▂▁▁▁▁▁


## 4. 모델 추론하기

- test data를 사용하여 모델의 성능을 확인합니다.

In [5]:
# ===============================
# 5. 모델 추론하기
# ===============================

def prepare_test_dataset(config, preprocessor: Preprocess, tokenizer):
    """
    test.csv → tokenization까지 완료한 Dataset 반환
    """
    test_file_path = os.path.join(config["general"]["data_path"], "test.csv")
    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data["fname"]

    print("-" * 120)
    print(f"test_data[0]:\n{test_data['dialogue'][0]}")
    print("-" * 120)

    encoder_input_test, decoder_input_test = preprocessor.make_input(
        test_data, is_test=True
    )
    print("✅ Load test data complete")

    test_tokenized_encoder_inputs = tokenizer(
        encoder_input_test,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config["tokenizer"]["encoder_max_len"],
        return_token_type_ids=False,
    )

    test_dataset = DatasetForInference(
        test_tokenized_encoder_inputs, test_id, len(encoder_input_test)
    )
    print("✅ Make test dataset complete")

    return test_data, test_dataset


def load_lora_model_for_inference(config, device):
    """
    추론용: base KoBART + 학습된 LoRA adapter 로드
    """
    base_model_name = config["general"]["model_name"]
    lora_path = config["inference"]["ckt_path"]

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.add_special_tokens(
        {"additional_special_tokens": config["tokenizer"]["special_tokens"]}
    )

    # base model
    base_model = BartForConditionalGeneration.from_pretrained(base_model_name)
    base_model.resize_token_embeddings(len(tokenizer))

    # LoRA adapter 로드
    model = PeftModel.from_pretrained(base_model, lora_path)
    model.to(device)
    model.eval()

    print(f"✅ Loaded LoRA model from: {lora_path}")
    return model, tokenizer


def inference(config):
    """
    학습된 LoRA 모델로 test.csv 요약 생성 및 output.csv 저장
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("-" * 10, f"device : {device}", "-" * 10)
    print(torch.__version__)

    generate_model, tokenizer = load_lora_model_for_inference(config, device)

    preprocessor = Preprocess(
        config["tokenizer"]["bos_token"], config["tokenizer"]["eos_token"]
    )
    test_data, test_dataset = prepare_test_dataset(
        config, preprocessor, tokenizer
    )

    dataloader = DataLoader(
        test_dataset, batch_size=config["inference"]["batch_size"]
    )

    summary_list = []
    text_ids = []

    with torch.no_grad():
        for item in tqdm(dataloader):
            input_ids = item["input_ids"].to(device)
            attention_mask = item["attention_mask"].to(device)
            text_ids.extend(item["ID"])

            generated_ids = generate_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                no_repeat_ngram_size=config["inference"]["no_repeat_ngram_size"],
                early_stopping=config["inference"]["early_stopping"],
                max_length=config["inference"]["generate_max_length"],
                num_beams=config["inference"]["num_beams"],
                min_length=25,          # 요약이 너무 짧아지는 것 방지
                length_penalty=1.1,     # 살짝 길이 패널티 부여
            )

            for ids in generated_ids:
                result = tokenizer.decode(ids, skip_special_tokens=False)
                summary_list.append(result)

    # 1차: 스페셜 토큰 제거
    remove_tokens = config["inference"]["remove_tokens"]
    preprocessed_summary = clean_generated_summary(summary_list, remove_tokens)

    # 2차: 반복 N-gram 제거
    preprocessed_summary = clean_repetition_in_summaries(
        preprocessed_summary, n=3
    )

    output = pd.DataFrame(
        {
            "fname": test_data["fname"],
            "summary": preprocessed_summary,
        }
    )

    result_path = config["inference"]["result_path"]
    os.makedirs(result_path, exist_ok=True)
    save_path = os.path.join(result_path, "output.csv")
    output.to_csv(save_path, index=False)

    print(f"✅ Saved inference result to: {save_path}")
    return output


if __name__ == "__main__":
    output = inference(loaded_config)
    output.head()


---------- device : cuda ----------
2.1.2+cu118


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ Loaded LoRA model from: ./lora_epoch_20
------------------------------------------------------------------------------------------------------------------------
test_data[0]:
#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. 
#Person2#: 네, 말씀하세요... 
#Person1#: 이걸 오늘 오후까지 모든 직원들에게 사내 메모로 보내야 해요. 준비됐나요? 
#Person2#: 네, 말씀하세요. 
#Person1#: 모든 직원에게 알립니다... 즉시 발효되어 모든 사내 통신은 이메일과 공식 메모로만 제한됩니다. 근무 시간 동안 즉시 메시지 프로그램 사용은 금지됩니다. 
#Person2#: 이 정책이 사내 통신에만 적용되나요, 아니면 외부 통신에도 해당되나요? 
#Person1#: 이는 모든 통신에 적용됩니다. 사무실 내 직원 간 통신 뿐만 아니라 외부 통신도 해당됩니다. 
#Person2#: 하지만 많은 직원들이 고객과 소통하려고 즉시 메시지를 사용합니다. 
#Person1#: 통신 방법을 바꿔야 할 것입니다. 이 사무실에서는 즉시 메시지를 사용하는 것을 원하지 않습니다. 너무 많은 시간이 낭비됩니다! 이제 계속해서 메모를 작성해 주세요. 어디까지 했죠? 
#Person2#: 내외부 통신에 적용됩니다. 
#Person1#: 네. 즉시 메시지를 계속 사용하면 경고 후 시정 조치가 이루어지며, 두 번째 위반 시 해고될 수 있습니다. 이번 정책에 관한 질문은 부서장에게 문의하세요. 
#Person2#: 그게 다인가요? 
#Person1#: 네. 오늘 오후 4시까지 이 메모를 작성하고 배포해주세요.
-------------------------------------------------------------------------------------------------------------------

100%|██████████| 16/16 [00:24<00:00,  1.56s/it]

✅ Saved inference result to: ./prediction/output.csv
